# Creating the Lumia eCommerce Database


In [ ]:
# install the required sql magic packages
!pip install sqlalchemy
!pip install ipython-sql
!pip install pymysql 

## Setting up Severless database engine 

In [2]:
# import the necessary packages
import sqlite3
from sqlalchemy import create_engine
import pandas as pd


In [4]:
# connect sqlite database (it will be created if it doesnt exist)

# Load the SQL extensions
%reload_ext sql

# Create engine 
engine = create_engine("sqlite:///lumia_ecommerce.db")
%config SQL.conn_name = 'engine'

conn = sqlite3.connect("lumia_ecommerce.db")
cursor = conn.cursor()

# Connect to the SQLite database
%sql sqlite:///lumia_ecommerce.db
# Data Defination Language
# create tables 
cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
               product_id INTEGER PRIMARY KEY,
               product_name TEXT NOT NULL,
               price DECIMAL NOT NULL,
               category TEXT NOT NULL
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS customers(
               customer_id INTEGER PRIMARY KEY,
               name TEXT NOT NULL,
               email TEXT 
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS orders(
               order_id INTEGER PRIMARY KEY,
               customer_id INTEGER NOT NULL,
               order_date DATE NOT NULL,
               FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS order_items(
               order_item_id INTEGER PRIMARY KEY,
               order_id INTEGER NOT NULL,
               product_id INTEGER NOT NULL,
               quantity INTEGER NOT NULL,
               FOREIGN KEY (order_id) REFERENCES orders(order_id),
               FOREIGN KEY (product_id) REFERENCES products(product_id)

);
''')

#### this can equally work 

In [ ]:
from sqlalchemy import create_engine

# Load the SQL extensions
%load_ext sql

# Create engine to connect to the SQLite database
engine = create_engine("sqlite:///lumia_ecommerce.db")

# Connect to the SQLite database using the %sql magic command
%sql sqlite:///lumia_ecommerce.db

# Create tables using SQL

# Table: products
%%sql CREATE TABLE IF NOT EXISTS products (
               product_id INTEGER PRIMARY KEY,
               product_name TEXT NOT NULL,
               price DECIMAL NOT NULL,
               category TEXT NOT NULL
)

# Table: customers
%%sql CREATE TABLE IF NOT EXISTS customers (
               customer_id INTEGER PRIMARY KEY,
               name TEXT NOT NULL,
               email TEXT 
)

# Table: orders
%%sql CREATE TABLE IF NOT EXISTS orders (
               order_id INTEGER PRIMARY KEY,
               customer_id INTEGER NOT NULL,
               order_date DATE NOT NULL,
               FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
)

# Table: order_items
%%sql CREATE TABLE IF NOT EXISTS order_items (
               order_item_id INTEGER PRIMARY KEY,
               order_id INTEGER NOT NULL,
               product_id INTEGER NOT NULL,
               quantity INTEGER NOT NULL,
               FOREIGN KEY (order_id) REFERENCES orders(order_id),
               FOREIGN KEY (product_id) REFERENCES products(product_id)
)


In [5]:
%%sql # this converts the cell to SQL cell
SELECT name FROM sqlite_master WHERE type in ('table', 'view') AND name NOT LIKE 'sqlite_%' ORDER BY 1

 * sqlite:///lumia_ecommerce.db
Done.


name
customers
order_items
orders
products


In [10]:
%%sql
SELECT * FROM customers;


 * sqlite:///lumia_ecommerce.db
Done.


customer_id,name,email


In [11]:
# function to load data into the table 
def load_data_to_table(csv_path, table_name):
    df = pd.read_csv(csv_path)
    df.to_sql(table_name, conn, if_exists='append', index=False)



In [16]:
# calling the function for each table with their relevant csv files and relative path 
load_data_to_table('../data/raw_datasets/products.csv', 'products')
load_data_to_table('../data/raw_datasets/customers.csv', 'customers')
load_data_to_table('../data/raw_datasets/orders.csv', 'orders')
load_data_to_table('../data/raw_datasets/order_items.csv', 'order_items')

# Data manipulations using DML



### INSERT


In [17]:
%%sql
INSERT INTO products (product_name, price, category) 
VALUES ('LG TV', 349.99, 'electronics')

 * sqlite:///lumia_ecommerce.db
1 rows affected.


[]

In [20]:
%%sql
SELECT * FROM products 
WHERE product_name LIKE '%TV'

 * sqlite:///lumia_ecommerce.db
Done.


product_id,product_name,price,category
1001,LG TV,349.99,electronics


In [26]:
# using python
result = cursor.execute('''
SELECT * FROM products LIMIT 5;
''')
result.fetchall()

[(1, 'Product 1', 193.52, 'Fashion'),
 (2, 'Product 2', 475.85, 'Home & Kitchen'),
 (3, 'Product 3', 368.68, 'Toys'),
 (4, 'Product 4', 303.34, 'Electronics'),
 (5, 'Product 5', 86.45, 'Toys')]

### UPDATE

In [33]:
%%sql
UPDATE customers SET email = 'ridwanclouds@ridwanclouds.com'
WHERE name = 'Customer1';

 * sqlite:///lumia_ecommerce.db
1 rows affected.


[]

### DELETE

In [35]:
%%sql
DELETE FROM orders
WHERE order_id = 1;

 * sqlite:///lumia_ecommerce.db
1 rows affected.


[]

### Data Query Language

In [42]:
%%sql
SELECT COUNT(*)
FROM orders
WHERE order_date BETWEEN '2022-01-01' AND '2022-12-01';

 * sqlite:///lumia_ecommerce.db
Done.


COUNT(*)
139


### Applying aggregate fucntions


* Calculating sales per category

In [48]:
%%sql
SELECT  category, SUM(price * quantity) as 'Total Sales'
FROM order_items as oi, products as p
WHERE oi.product_id = p.product_id
GROUP BY category;

 * sqlite:///lumia_ecommerce.db
Done.


category,Total Sales
Books,819475.7699999996
Electronics,782752.7699999994
Fashion,794716.4500000014
Home & Kitchen,662919.4899999999
Toys,716282.3800000007


* Calculating highest selling product

In [51]:
%%sql
SELECT product_id, SUM(quantity) as 'Total Quantity'
FROM order_items
GROUP BY product_id
ORDER BY 'Total Quantity' DESC
LIMIT 5;

 * sqlite:///lumia_ecommerce.db
Done.


product_id,Total Quantity
1000,14
999,17
998,6
997,14
996,14


In [59]:
%%sql
SELECT COUNT(*) AS "Total Electronics Products"
FROM products
WHERE category = 'Electronics';

 * sqlite:///lumia_ecommerce.db
Done.


Total Electronics Products
193


#### HAVING clause


In [60]:

'''
SELECT columns
FROM tables
GROUP BY  columns
HAVING boolean expressions
'''

'\nSELECT columns\nFROM tables\nGROUP BY  columns\nHAVING boolean expressions\n'

* Categories with more than 1000 sales

In [85]:
%%sql
SELECT p.category, COUNT(o.order_id) as 'orders_count'
FROM orders o, products p, order_items oi
WHERE o.order_id = oi.order_id 
    AND
    oi.product_id = p.product_id
GROUP BY p.category
HAVING orders_count >1100;

    

 * sqlite:///lumia_ecommerce.db
Done.


category,orders_count
Books,1302
Electronics,1181
Fashion,1245
Toys,1251


* Average product price per category ordered by average price 

In [80]:
%%sql
SELECT category, AVG(price) as Average_Price
FROM products
GROUP BY category
ORDER BY Average_Price DESC;

 * sqlite:///lumia_ecommerce.db
Done.


category,Average_Price
Fashion,261.4665384615386
Electronics,256.1550259067357
Home & Kitchen,253.98537572254324
Books,245.67896226415084
Toys,235.41691588785045


#### JOINS 
- Inner, Left, Right, Outer, Cross

------------------------------------------------------

- top 5 products ordered

In [91]:
%%sql
SELECT p.product_name, o.order_date
FROM products p
INNER JOIN order_items oi ON 
    p.product_id = oi.product_id
INNER JOIN orders o ON
    o.order_id = oi.order_id
ORDER BY order_date DESC LIMIT 5;


 * sqlite:///lumia_ecommerce.db
Done.


product_name,order_date
Product 949,2024-01-27 21:45:33
Product 877,2024-01-27 21:45:33
Product 854,2024-01-27 21:45:33
Product 612,2024-01-22 15:55:13
Product 602,2024-01-22 15:55:13


In [96]:
%%sql
SELECT *
FROM customers c
LEFT JOIN orders o ON 
    c.customer_id = o.customer_id
LIMIT 3;

 * sqlite:///lumia_ecommerce.db
Done.


customer_id,name,email,order_id,customer_id_1,order_date
1,Customer1,ridwanclouds@ridwanclouds.com,1331,1,2018-08-13 02:38:21
2,Customer2,Customer2@gmail.com,26,2,2014-11-04 05:41:36
2,Customer2,Customer2@gmail.com,132,2,2016-02-17 07:12:21


### UNION
- combine two or more 'SELECT' statements into a single result set, provided same number of columns, columns are in order, and compatible data types,  in the SELECT statements you are trying to unify. 
- UNION ALL will remove duplicates


In [105]:
%%sql
SELECT customer_id, name, email
FROM customers
UNION
SELECT product_id, product_name, category
FROM products
LIMIT 6;


 * sqlite:///lumia_ecommerce.db
Done.


customer_id,name,email
1,Customer1,ridwanclouds@ridwanclouds.com
1,Product 1,Fashion
2,Customer2,Customer2@gmail.com
2,Product 2,Home & Kitchen
3,Customer3,Customer3@gmail.com
3,Product 3,Toys


### Concatenation | |

In [112]:
%%sql
SELECT name || ', ' || email as 'Full_Details'
FROM customers
LIMIT 3;

 * sqlite:///lumia_ecommerce.db
Done.


Full_Details
"Customer1, ridwanclouds@ridwanclouds.com"
"Customer2, Customer2@gmail.com"
"Customer3, Customer3@gmail.com"
